In [2]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.style.use('seaborn-poster')
from datetime import datetime as dt
from datetime import timedelta
import glob
    
from Reff_functions import *
from Reff_constants import *

In [3]:
#Code taken from read_in_cases from Reff_functions. Preprocessing was not helpful for this situation.

case_file_date='31Jul'
path = "../data/COVID-19 UoM "+case_file_date+"*.xlsx"

for file in glob.glob(path):
    df_NNDSS = pd.read_excel(file,
                       parse_dates=['SPECIMEN_DATE','NOTIFICATION_DATE','NOTIFICATION_RECEIVE_DATE','TRUE_ONSET_DATE'],
                       dtype= {'PLACE_OF_ACQUISITION':str})
    df_NNDSS.PLACE_OF_ACQUISITION.fillna('00038888',inplace=True) #Fill blanks with simply unknown

   # df_NNDSS['date_inferred'] = df_NNDSS.TRUE_ONSET_DATE
  #  df_NNDSS.loc[df_NNDSS.TRUE_ONSET_DATE.isna(),'date_inferred'] = df_NNDSS.loc[df_NNDSS.TRUE_ONSET_DATE.isna()].NOTIFICATION_DATE - timedelta(days=5)
  #  df_NNDSS.loc[df_NNDSS.date_inferred.isna(),'date_inferred'] = df_NNDSS.loc[df_NNDSS.date_inferred.isna()].NOTIFICATION_RECEIVE_DATE - timedelta(days=6)    
df_NNDSS['imported'] = df_NNDSS.PLACE_OF_ACQUISITION.apply(lambda x: 1 if x[-4:]=='8888' and x != '00038888' else 0)
df_NNDSS['local'] = 1 - df_NNDSS.imported

df_interim = df_NNDSS[['NOTIFICATION_DATE','STATE','imported','local']] 
# Importantly, imported and local are indicator variables in df_interim.

#df_state = df_NNDSS[['NOTIFICATION_DATE','STATE','imported','local']].groupby(['STATE','NOTIFICATION_DATE']).sum()

In [4]:
df_interim = df_interim[~df_interim.NOTIFICATION_DATE.isna()] #Get rid of non-existent dates.

In [5]:
#Filter out territories
df_linel = df_interim[(df_interim['STATE']!='NT') & (df_interim['STATE']!='ACT')]
df_linel.head()
#df_linel['STATE'].unique()

,NOTIFICATION_DATE,STATE,imported,local
113,2020-07-30,NSW,0,1
114,2020-07-30,NSW,0,1
115,2020-07-30,NSW,0,1
116,2020-07-29,NSW,0,1
117,2020-07-30,NSW,0,1


## Part 1: Inferring infection dates
$\Lambda$ depends on the infection date (ID), while the data contains the notification date (ND). We obtain ID through the following relationship:
$$
ID = ND - reporting\_delay - incubation\_period.
$$



A gamma distribution was fitted to case data using the MLE algorithm to produce distributions for reporting delay and incubation period. 

In [6]:
#gamma draws take arguments (shape, scale)
def create_inf_dates(notification_dates, mean_rd=5.47, sd_rd=4.04,
                    mean_inc=2.0, sd_inc=1.41, nreplicates=1):

    nsamples = notification_dates.shape[0]

#    DEFINE DELAY DISTRIBUTION
#     mean_rd = 5.47
#     sd_rd = 4.04
    scale_rd = mean_rd/(sd_rd)**2
    shape_rd = mean_rd/scale_rd

    # DEFINE INCUBATION PERIOD DISTRIBUTION
#     mean_inc = 2.0
#     sd_inc = 1.41
    scale_inc = mean_inc/(sd_inc)**2
    shape_inc = mean_inc/scale_inc
    
    #Draw from distributions
    inc_period = np.random.gamma(shape_inc, scale_inc, size = (nsamples))
    rep_delay = np.random.gamma(shape_rd, scale_rd, size = (nsamples))
    
    #infection date is id_nd_diff days before notification date.
    id_nd_diff = inc_period + rep_delay
    
    #Minutes aren't included in df. Take the ceiling because the day runs from 0000 to 2359.
    whole_day_diff = np.ceil(id_nd_diff) 
    
    #Subtract days off of notification dates.
    infection_dates = notification_dates - whole_day_diff.astype('timedelta64[D]')
    
    return(infection_dates)

In [7]:
#Add infection dates to original data frame.
df_interim.loc[:,'infection_date'] = create_inf_dates(df_interim['NOTIFICATION_DATE'])

In [8]:
df_interim.head()

,NOTIFICATION_DATE,STATE,imported,local,infection_date
0,2020-07-09,ACT,0,1,2020-07-01
1,2020-07-08,ACT,0,1,2020-07-01
2,2020-07-09,ACT,0,1,2020-06-29
3,2020-07-08,ACT,0,1,2020-07-03
4,2020-07-08,ACT,0,1,2020-07-01


In [9]:
df_state = df_interim[['infection_date','STATE','imported','local']].groupby(['STATE','infection_date']).sum()

In [10]:
df_state.tail()

imported  local
STATE infection_date                 
WA    2020-05-02             1      0
      2020-05-17             2      0
      2020-05-19             1      0
      2020-05-20             1      0
      2020-05-23             1      0

In [22]:
df_total = df_state
df_total['total'] = df_total['imported'] + df_total['local']
df_total = df_total.drop('imported',1)
df_total.head()

local  total_cases  total
STATE infection_date                           
ACT   2020-03-04          1            1      1
      2020-03-09          1            2      2
      2020-03-10          1            3      3
      2020-03-11          0            2      2
      2020-03-12          0            1      1

## Part 2: Calculating Lambda